In [1]:
from pm4py.objects.log.importer.xes import factory as xes_importer
from pm4py.objects.log.importer.csv import factory as csv_importer

from pm4py.algo.discovery.dfg import factory as dfg_factory

import pandas as pd

import timeit
import os
import datetime

In [2]:
method = 'pm4py'

# if you use kubernetes, you can map the dicingDate to the arguments.
# dicingDate = os.getenv('DICING_DATE')
dicingDate = '2015-07-01'

In [ ]:
file = '/data/BPI2016_Clicks_NOT_Logged_In.csv'

In [ ]:
experiment2_file = '/data/experiment2.csv'

In [ ]:
result_columns = ['Analysis', 'Number Of Events', 'Loading Time', 'DFG Calculation Time', 'Total Time']

In [ ]:
def getDFG(result, method, dicingDate, result_columns):
    dicingDate = datetime.datetime.strptime(dicingDate, '%Y-%m-%d')
    queryDate = dicingDate
    queryDate = queryDate.strftime("%Y-%m-%d")
    
    time_start = timeit.default_timer()

    log = pd.read_csv(file, sep=';', encoding='latin-1', usecols=["SessionID", "PAGE_NAME", "TIMESTAMP"])
    log = log.rename(columns={"SessionID": "case:concept:name", "PAGE_NAME": "concept:name", "TIMESTAMP": "time:timestamp"})

    log["time:timestamp"] =  pd.to_datetime(log["time:timestamp"])

    log = log[log['time:timestamp'].dt.strftime('%Y-%m-%d') <= queryDate]

    numberOfEvents = len(log.index)
    
    time_log = timeit.default_timer()

    dfg = dfg_factory.apply(log)

    time_stop = timeit.default_timer()

    Loading_Time = time_log-time_start
    DFG_Calculation_Time = time_stop-time_log
    Total_Time = time_stop-time_start

    new_row = pd.DataFrame([[method, numberOfEvents, Loading_Time, DFG_Calculation_Time, Total_Time]]
                           , columns=result_columns)

    result = pd.concat([result, new_row])
    
    return result

In [7]:
if os.path.isfile(experiment2_file):
    result = pd.read_csv(experiment2_file)
else:
    result = pd.DataFrame(columns=result_columns)

In [8]:
result = getDFG(result, method, dicingDate, result_columns)

In [11]:
result = result.sort_values(by=result_columns)
result.to_csv(experiment2_file, index = False)